In [1]:
from tvDatafeed import TvDatafeed, Interval
import config_api
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import yfinance as yf


from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [11]:
data = yf.download(['BTC-USD', 'BNB-USD', 'XRP-USD', 'ETH-USD', 'ADA-USD', 'DOGE-USD'], start= '2019-01-01', end= '2022-01-01')

[*********************100%***********************]  6 of 6 completed


In [12]:
df = data.Close.copy()

In [14]:
df = df[['ADA-USD', 'BNB-USD', 'BTC-USD', 'XRP-USD', 'DOGE-USD', 'ETH-USD']]

In [15]:
### define base model

def baseline_model():
    # initiate model
    model = Sequential()
    model.add(Dense(1, input_dim=5))
    # compile model
    model.compile(loss='mean_squared_error')
    model.summary()
    return model

In [16]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
# evaluate the baseline model, use 10 fold cross validataion

estimator = KerasRegressor(build_fn= baseline_model, epochs= 100, batch_size=5, verbose=0)
kfold = KFold(n_splits= 10)
results =  cross_val_score(estimator, X, y, cv=kfold)

In [21]:
# evaluate the baseline model, use 10 fold cross validataion 
# and standardized dataset
estimator = []
estimator.append(('standardize', StandardScaler()))
estimator.append(('mlp', KerasRegressor(build_fn= baseline_model, epochs= 100, batch_size=5, verbose=0)))

#### pipeline is sklearn framework,
#### to perform the standardization during the model evaluation procress, with each fold cross validation
#### this ensure that there is no data leakage from each testset cross validtaion fold into the training data
pipeline = Pipeline(estimator)
kfold = KFold(n_splits= 10)
results =  cross_val_score(pipeline, X, y, cv=kfold)

<ipython-input-21-11ebfe5fc0f8>:5: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  estimator.append(('mlp', KerasRegressor(build_fn= baseline_model, epochs= 100, batch_size=5, verbose=0)))


Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 1)                 6         
                                                                 
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 1)                 6         
                                                                 
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)  

In [22]:
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Baseline: -2761912.32 (4593101.30) MSE


In [23]:
print(results)

[   -29435.41796875    -70136.4375        -45916.76953125
    -46122.14453125    -53729.26171875   -145160.53125
   -774926.125       -5021506.          -6733132.5
 -14699058.        ]
